In [3]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize

In [4]:
#The following code takes a list of tickers from a csv file, a start date, and an end date. It then generates a dataframe with the daily returns
#of every portfolio within the specified start and end date. It also does this with the two market indexes.

#Another function takes the portfolio as a parameter, and flags every month (in a print function) in which there are 
#less than 18 rows available for any stock. The function then removes that month's data.

#A third function generates a parallel dataframe that shows the percent daily returns of each stock. The percent returns of the two indices are
#calculated and another column showing the final "index" average is made.

In [5]:
START_DATE = '2023-11-25'
END_DATE = '2024-11-19'

In [6]:

ticker_lst = pd.read_csv('Ticker_list.csv')["Ticker"].tolist()
print(ticker_lst)
index_lst = ["XIU.TO","^GSPC"]

exch_rate = yf.download("CADUSD=x",start="2024-11-20", end="2024-11-21")["Close"].iloc[0] #MUST BE CHANGED TO PULL NOVEMBER 22ND

#Takes a yf.Ticker "ticker", checks if the ticker is in canadian dollars. If so, it will just do a simple API pull for the price history.
#If the price is listed in USD, a conversion operation will simply be applied to each price.
def yfin_pull_convert_USD(ticker):
    listed_currency = ticker.fast_info["currency"]
    if listed_currency == "CAD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
    if listed_currency == "USD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"] * (1/exch_rate)
    else: print("ticker currency is not in USD or CAD error")

#Test cases
#display(yf.Ticker("NVDA").history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
#yfin_pull_convert_USD(yf.Ticker("NVDA"))

#Main function to store the stock prices of the ticker
def gen_tickers(tlist, ilist):
    ret_dataframe = pd.DataFrame()
    for i_str in ilist:
        index_ticker = yf.Ticker(i_str)
        ret_dataframe[i_str] = yfin_pull_convert_USD(index_ticker)
    
    for t_str in tlist:
        ticker = yf.Ticker(t_str)
        ret_dataframe[t_str] = yfin_pull_convert_USD(ticker)
    return ret_dataframe

#Function call: stores a dataframe of index values and stock values
stock_values = gen_tickers(ticker_lst,index_lst)

FileNotFoundError: [Errno 2] No such file or directory: 'Ticker_list.csv'

In [ ]:
market_label = "Market Returns"
#Function that transforms a list of values into a list of percent returns.
#Also adds a column of the simple average returns of the index tickers.
def convert_pct_returns(stock_value_df):
    ret_dataframe = stock_value_df.interpolate().pct_change()
    ret_dataframe.dropna(inplace=True)
    ret_dataframe[market_label] = ret_dataframe[index_lst].mean(axis=1)
    df_col_order = index_lst + ["Market Returns"] + ticker_lst
    ret_dataframe = ret_dataframe[df_col_order]
    return ret_dataframe

stock_returns = convert_pct_returns(stock_values)

display(stock_returns)

In [ ]:
#Creating the correlation matrix: takes a dataframe of values and a string of columns to drop to make the correlation matrix
def correl(data, dropvalue):
    data_marketdrop = data.drop(labels=dropvalue, axis=1)
    ret_corr = data_marketdrop.corr()
    return ret_corr

stock_correlations = correl(stock_returns, [market_label] + index_lst)
#Apply a format to the DataFrame
display(stock_correlations.style.background_gradient(cmap='RdYlGn_r'))


In [ ]:


#finding the ticker coordinates of the lowest correlation value (AI)
def low_correl_ticker_pairs(correlation_matrix):
    if correlation_matrix.shape[0] > 1:
        correl_pairs = correlation_matrix.unstack()
        correl_pairs = correl_pairs[correl_pairs.index.get_level_values(0) != correl_pairs.index.get_level_values(1)]
        lowest_corr_pair = correl_pairs.idxmin()
        lowest_corr_value = correl_pairs.min()
        return list(lowest_corr_pair)
    else:
        remaining_column_ticker = correlation_matrix.columns[0]
        return [remaining_column_ticker]

#recursing through the entire correlation matrix to extract every correlation pair
def corr_pair_extract(correl_matrix):
    #creating a list to store low-correlation pairs into
    low_corr_pairs = []
    while correl_matrix.shape[0] > 0:
        pair = low_correl_ticker_pairs(correl_matrix)
        #removing the tickers after they are packaged into a correlation pair
        if isinstance(pair, str):
            ticker1 = pair
            correl_matrix = correl_matrix.drop(ticker1, axis=0)
            correl_matrix = correl_matrix.drop(ticker1, axis=1)
        else: 
            low_corr_pairs.append(pair)
            correl_matrix = correl_matrix.drop(pair, axis=0)
            correl_matrix = correl_matrix.drop(pair, axis=1)
    return low_corr_pairs

#running the correlation pairing function and storing it in variable "stock pairs"
stock_pairs = corr_pair_extract(stock_correlations)
print(stock_pairs)

In [ ]:
#Arbitrarily assign a seed to keep results consistent
random.seed(21152764)

#This part creates lists of stocks, ranging from sizes 12 to 24, 5 of each size, which randomly selects stocks.
def make_port_list(stock_pairs):
    ret_list = []
    #iterates through the minimum selectable stock pairs (6) and the maximum (length of the stock pairs)
    for i in range(6, len(stock_pairs)):
        #simple iteration of 5 times
        for j in range(0,5):
            #generates a random list of indexes to pick from the stock pairs
            rand_list = random.sample(range(0,len(stock_pairs)-1), i)
            portfolio = []
            for k in rand_list:
                portfolio += stock_pairs[k]
            ret_list.append(portfolio)
    return ret_list

port_list = make_port_list(stock_pairs)
display(port_list)
len(port_list)